In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

np.random.seed(42)
X, y = make_moons(n_samples=1000, noise=0.2)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

def one_hot(y, k=2):
    Y = np.zeros((y.shape[0], k))
    Y[np.arange(y.shape[0]), y] = 1
    return Y
Y_train = one_hot(y_train, 2)
Y_test = one_hot(y_test, 2)


In [1]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
def dsigmoid(a):
    return a*(1-a)
def tanh(z):
    return np.tanh(z)
def dtanh(a):
    return 1-a**2
def relu(z):
    return np.maximum(0, z)
def drelu(z):
    return (z>0).astype(float)

ACTS = {'sigmoid': (sigmoid, dsigmoid), 'tanh': (tanh, dtanh), 'relu': (relu, drelu)}

def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    exp = np.exp(z)
    return exp/np.sum(exp, axis=1, keepdims=True)
def cross_entropy(p, Y):
    eps=1e-9
    return -np.mean(np.sum(Y*np.log(p+eps), axis=1))
